In [8]:
import copy, numpy as np


np.random.seed(2)

In [14]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x)) if x >= 0 else np.exp(x) / (1.0 + np.exp(x))

def sigmoid2(x):
    return 1.0 / (1.0 + np.exp(-x))

def sigmoid3(x):
    return 1.0 / (1.0 + np.exp(-np.clip(x, 255, -255)))

x = 999
print("sigmoid(%.15f): %.15f" % (x, sigmoid(x)))
print("sigmoid2(%.15f): %.15f" % (x, sigmoid2(x)))
print("sigmoid3(%.15f): %.15f" % (x, sigmoid3(x)))

print("sigmoid(%.15f): %.15f" % (-x, sigmoid(-x)))
print("sigmoid2(%.15f): %.15f" % (-x, sigmoid2(-x)))
print("sigmoid3(%.15f): %.15f" % (-x, sigmoid3(-x)))


sigmoid(999.000000000000000): 1.000000000000000
sigmoid2(999.000000000000000): 1.000000000000000
sigmoid3(999.000000000000000): 0.000000000000000
sigmoid(-999.000000000000000): 0.000000000000000
sigmoid2(-999.000000000000000): 0.000000000000000
sigmoid3(-999.000000000000000): 0.000000000000000


<ipython-input-14-408544179a7e>:5: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))
